# Setup Paths

In [1]:
import os

def setup_paths(dir_location, is_original_data):
    if dir_location.lower() == 'internal':
        base_path = r'C:\Senior_Design'
    elif dir_location.lower() == 'external':
        base_path = r'D:\Senior_Design'
    elif dir_location.lower() == 'cloud':
        base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
    elif dir_location.lower() == 'refine':
        base_path = r'D:\Darren\Files'
    else:
        raise ValueError('Invalid directory location type')
    
    base_gt_path = os.path.join(base_path, 'database')
    if is_original_data:
        gt_path = os.path.join(base_gt_path, 'orignal_dataset', 'segmented', 'tiff')
    else:
        gt_path = os.path.join(base_gt_path, 'tablet_dataset', 'segmented', 'tiff')
    
    binary_path = gt_path.replace('segmented', 'binary')  
    if not os.path.isdir(binary_path):
        os.makedirs(binary_path)
    bordercore_path = gt_path.replace('segmented', 'bordercore')  
    if not os.path.isdir(bordercore_path):
        os.makedirs(bordercore_path)  
    instance_path = gt_path.replace('segmented', 'instance_temp')
    if not os.path.isdir(instance_path):
        os.makedirs(instance_path)

    print('Paths set')
    return gt_path, binary_path, bordercore_path, instance_path

# Extract Pixel Values from Ground Truth TIFF Stack

In [2]:
import numpy as np
import tifffile as tiff
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

def extract_unique_pixel_values(folder_path):
    """Extract unique pixel values and their counts from a TIFF stack in a folder."""
    pixel_values = []
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            img = tiff.imread(os.path.join(folder_path, filename))
            pixel_values.append(img.flatten())  # Flatten to a 1D array
    
    # Concatenate all pixel values and get unique values with counts
    all_pixels = np.concatenate(pixel_values)
    unique_values, counts = np.unique(all_pixels, return_counts=True)
    
    return unique_values, counts  # Return both unique values and their frequencies

def highlight_pixel_value_one_image(folder_path, target_value):
    """Display an image where only a specific pixel value is shown, using the middle file in the folder."""
    tiff_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tif') or f.endswith('.tiff')])
    
    if not tiff_files:
        raise FileNotFoundError("No TIFF files found in the folder.")
    
    # Select the middle file
    middle_file = tiff_files[len(tiff_files) // 2]
    img = tiff.imread(os.path.join(folder_path, middle_file))  # Load the middle file
    
    # Create a mask where only the target pixel value is highlighted
    highlighted_img = np.where(img == target_value, target_value, 0).astype(np.uint8)
    
    plt.imshow(highlighted_img, cmap="gray")
    plt.axis("off")
    plt.show()  # Display without saving

def load_tiff_stack_from_dir(directory):
    """Loads a stack of TIFF images from a directory into a 3D NumPy array."""
    tiff_files = sorted([f for f in os.listdir(directory) if f.lower().endswith('.tiff') or f.lower().endswith('.tif')])
    
    if not tiff_files:
        raise FileNotFoundError("No TIFF files found in the directory.")

    stack = [tiff.imread(os.path.join(directory, f)) for f in tiff_files]
    return np.stack(stack, axis=0)  # Stack images along the first dimension (depth)

In [ ]:
tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\2_Tablet'
pixel_values, counts = extract_unique_pixel_values(tiff_path)
print(pixel_values)
print(counts)

In [ ]:
tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\2_Tablet'
highlight_pixel_value_one_image(tiff_path, target_value=255)

In [ ]:
tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\4_GenericD12'
pixel_values, counts = extract_unique_pixel_values(tiff_path)
print(pixel_values)
print(counts)

In [ ]:
tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\4_GenericD12'
highlight_pixel_value_one_image(tiff_path, target_value=182)

In [ ]:
tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\5_ClaritinD12'
pixel_values, counts = extract_unique_pixel_values(tiff_path)
print(pixel_values)
print(counts)

In [ ]:
tiff_path = r'd:\Darren\Files\database\tablet_dataset\segmented\tiff\5_ClaritinD12'
highlight_pixel_value_one_image(tiff_path, target_value=159)

# Semantic to Binary

In [ ]:
def semantic_to_binary(input_path, output_path, target_value):
    tiff_stack = load_tiff_stack_from_dir(input_path)
    
    # Create a mask where only the target pixel value is highlighted
    binary_mask = np.where(tiff_stack == target_value, 255, 0).astype(np.uint8)
    
    for i in tqdm(range(binary_mask.shape[0]), desc=f"Processing {os.path.basename(input_path)}", unit="slice"):
        binary_path = os.path.join(output_path, f"slice_{i:04d}.tiff")
        tiff.imwrite(binary_path, binary_mask[i])

In [ ]:
import os

dir_location = 'refine'
gt_path, binary_path, _, _ = setup_paths(dir_location, False)
# img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
#                '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
#                '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
# target_values = [255, 255, 255, 255, 255,
#                  182, 182, 182, 182, 182,
#                  159, 159, 159, 159, 159]
img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']
target_values = [255, 182, 159]
for img_name, target_value in zip(img_names, target_values):
    input_path = os.path.join(gt_path, img_name)
    output_path = os.path.join(binary_path, img_name)
    if not os.path.isdir(output_path):
        os.makedirs(output_path)
    semantic_to_binary(input_path, output_path, target_value)

# Binary to Border-Core

In [ ]:
import numpy as np
from pathlib import Path
import tifffile as tiff
from skimage.morphology import binary_erosion, dilation, ball, footprint_rectangle


def generate_bordercore(image, border_thickness=1): # Uses erosion and dilation to generate border-core.
    """
    Generate border pixels from 3D binary masks.
    
    Parameters:
        image (np.ndarray): 3D binary image (0s and 255s, or really 0 and any other positive value).
        border_thickness (int): Thickness of the border region.
        
    Returns:
        np.ndarray: Image with cores labeled as 255 and borders labeled as 127.
    """
    n_erosions = border_thickness

    # Ensure binary input (convert 255 to 1 for processing)
    binary_image = image > 0

    # Define 3D structuring elements
    erosion_kernel = ball(n_erosions)  # Spherical structuring element for erosion
    dilation_kernel = footprint_rectangle([2 * border_thickness + 1, 2 * border_thickness + 1, 2 * border_thickness + 1])  # Cubic structuring element for dilation
    # dilation_kernel = ball(border_thickness)

    # Apply 3D erosion
    eroded_image = binary_erosion(binary_image, erosion_kernel)

    # Apply 3D dilation
    dilated = dilation(eroded_image, dilation_kernel)

    # Assign labels: cores as 255, borders as 127, background as 0
    bordercore = np.zeros_like(image, dtype=np.uint8)
    bordercore[eroded_image] = 255  # Core pixels
    bordercore[dilated & ~eroded_image] = 127  # Border pixels

    return bordercore


# def generate_bordercore(image, border_thickness=1): # Uses for erosion only to generate border-core.
#     """
#     Generate border pixels from 3D binary masks.
    
#     Parameters:
#         image (np.ndarray): 3D binary image (0s and 255s, or really 0 and any other positive value).
#         border_thickness (int): Thickness of the border region.
        
#     Returns:
#         np.ndarray: Image with cores labeled as 255 and borders labeled as 127.
#     """
    
#     # Ensure binary input (convert 255 to 1 for processing)
#     binary_image = image > 0

#     # Define 3D structuring elements
#     erosion_kernel = ball(border_thickness)  # Spherical structuring element for erosion

#     # Apply 3D erosion
#     eroded_image = binary_erosion(binary_image, erosion_kernel)


#     # Assign labels: cores as 255, borders as 127, background as 0
#     bordercore = np.zeros_like(image, dtype=np.uint8)
#     bordercore[eroded_image] = 255  # Core pixels
#     bordercore[binary_image & ~eroded_image] = 127  # Border pixels

#     return bordercore


def process_tiff_stack_dir(input_dir: str, border_thickness=1):
    """
    Process all TIFF images in a directory as a single 3D volume.
    
    Parameters:
        input_dir (str): Directory containing TIFF images.
        border_thickness (int): Border Thickness for dilation.
        n_erosions (int): Number of erosion iterations.
    """
    input_path = Path(input_dir)
    output_path = Path(str(input_dir).replace('binary', 'bordercore'))
    output_path.mkdir(parents=True, exist_ok=True)

    # Get all TIFF files and sort them to maintain order
    tiff_files = sorted(input_path.glob("*.tiff"))
    if not tiff_files:
        raise ValueError("No TIFF files found in the directory.")

    # Read the entire directory as a 3D stack
    stack = np.array([tiff.imread(str(file)) for file in tiff_files], dtype=np.uint8)

    # Process the 3D volume
    processed_stack = generate_bordercore(stack, border_thickness)

    # Save each slice as an individual TIFF file
    for i, slice_ in enumerate(processed_stack):
        slice_path = os.path.join(output_path, f"slice_{i:04d}.tiff")
        tiff.imwrite(slice_path, slice_.astype(np.uint8))

    print(f"Processed 3D stack saved to: {output_path}")

In [ ]:
dir_location = 'refine'
_, binary_path, _, _ = setup_paths(dir_location, False)
# img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
#             '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
#             '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']

for img_name in img_names:
    input_path = os.path.join(binary_path, img_name)
    process_tiff_stack_dir(input_path, border_thickness=1)

# Border-Core to Instance

In [ ]:
import os
import numpy as np
import tifffile as tiff
import cc3d
import copy
from scipy.ndimage import label as nd_label
from skimage.morphology import footprint_rectangle, dilation, ball
from scipy.ndimage import distance_transform_edt
from tqdm import tqdm
import numpy_indexed as npi
from typing import Tuple, Type
from acvl_utils.miscellaneous.ptqdm import ptqdm


def process_tiff_dir(input_dir, output_dir, processes=None):
    os.makedirs(output_dir, exist_ok=True)
    border_core = load_tiff_stack_from_dir(input_dir)
    instances, _ = border_core2instance(border_core, processes=processes)
    for i in tqdm(range(instances.shape[0]), desc=f"Processing {os.path.basename(input_dir)}", unit="slice"):
        output_path = os.path.join(output_dir, f"slice_{i:04d}.tiff")
        tiff.imwrite(output_path, instances[i])
    print(f"Processing complete. Output saved to {output_dir}")

def border_core2instance(border_core: np.ndarray, dtype: Type = np.uint16, processes: int = None) -> Tuple[np.ndarray, int]:
    border_core_array = np.array(border_core)
    component_seg = cc3d.connected_components(border_core_array > 0).astype(dtype) # Original method
    instances = np.zeros_like(border_core, dtype=dtype)
    num_instances = 0
    props = {i: bbox for i, bbox in enumerate(cc3d.statistics(component_seg)["bounding_boxes"]) if i != 0}

    if processes is None or processes == 0:
        for label, bbox in tqdm(props.items(), desc="Border-Core2Instance"):
            filter_mask = component_seg[bbox] == label
            border_core_patch = copy.deepcopy(border_core[bbox])
            border_core_patch[filter_mask != 1] = 0
            instances_patch = border_core_component2instance_dilation(border_core_patch).astype(dtype)
            instances_patch[instances_patch > 0] += num_instances
            num_instances = max(num_instances, np.max(instances_patch))
            # patch_labels = np.unique(instances_patch)
            # patch_labels = patch_labels[patch_labels > 0]
            # for patch_label in patch_labels:
            #     instances[bbox][instances_patch == patch_label] = patch_label
            instances[bbox][instances_patch > 0] = instances_patch[instances_patch > 0] # SOOOOOOO MUCH FASTER THAN THE 4 LINES ABOVE!!!!!!!! Confirmed no impact on segmentation!

    else:
        border_core_patches = []
        for label, bbox in props.items():
            filter_mask = component_seg[bbox] == label
            border_core_patch = copy.deepcopy(border_core[bbox])
            border_core_patch[filter_mask != 1] = 0
            border_core_patches.append(border_core_patch)

        instances_patches = ptqdm(border_core_component2instance_dilation, border_core_patches, processes, desc="Border-Core2Instance")

        for index, (label, bbox) in enumerate(tqdm(props.items())):
            instances_patch = instances_patches[index].astype(dtype)
            instances_patch[instances_patch > 0] += num_instances
            num_instances = max(num_instances, int(np.max(instances_patch)))
            # patch_labels = np.unique(instances_patch)
            # patch_labels = patch_labels[patch_labels > 0]
            # for patch_label in patch_labels:
            #     instances[bbox][instances_patch == patch_label] = patch_label
            instances[bbox][instances_patch > 0] = instances_patch[instances_patch > 0] # SOOOOOOO MUCH FASTER THAN THE 4 LINES ABOVE!!!!!!!! Confirmed no impact on segmentation!

    return instances, num_instances

def border_core_component2instance_dilation(patch: np.ndarray, core_label: int = 255, border_label: int = 127) -> np.ndarray:
    '''The values used for core and border don't actually matter: 
       'num_instances = nd_label(patch == core_label, output=core_instances)' assigns unique instance labels where patch == core_label, regardless of whether core_label is 1, 255, or any other value.
       'border = patch == border_label' creates a binary mask (True for border, False elsewhere). Whether border_label is 2, 127, or any other value does not affect behavior.
       'dilated = dilation(core_instances, footprint_rectangle([3,3,3]))' expands the core_instances which are the instance labels (1-# of instances) and the background (0). Thus, all instances are "brighter" than the surrounding background (no border that is brighter/darker than core).
    '''
    # core_instances = np.zeros_like(patch, dtype=np.uint16)
    # num_instances = nd_label(patch == core_label, output=core_instances)
    core_instances = cc3d.connected_components(patch == core_label, connectivity=6) # cc3d.connected_components is better optimized than scipy.ndimage.label | Confirmed no impact on segmentation!
    num_instances = core_instances.max()
    
    if num_instances == 0:
        return patch
    
    # # Significiant bottleneck when patch size is big and has many particles in it (same reason as for the for loop that iterates through the patch labels)
    # patch, core_instances, num_instances = remove_small_cores(patch, core_instances, core_label, border_label) # Weird effect on segmentation...
    # # core_instances = np.zeros_like(patch, dtype=np.uint16)
    # # num_instances = nd_label(patch == core_label, output=core_instances)
    # core_instances = cc3d.connected_components(patch == core_label, connectivity=6) # cc3d.connected_components is better optimized than scipy.ndimage.label
    # num_instances = core_instances.max()
    # if num_instances == 0:
    #     return patch
    
    instances = copy.deepcopy(core_instances)
    border = patch == border_label
    while np.sum(border) > 0:
        dilated = dilation(core_instances, footprint_rectangle([3,3,3])) # Bottleneck when many instances
        # dilated = dilation(core_instances, ball(1)) # Bottleneck when many instances
        dilated[patch == 0] = 0
        diff = (core_instances == 0) & (dilated != core_instances)
        instances[diff & border] = dilated[diff & border]
        border[diff] = 0
        core_instances = dilated
    return instances


def remove_small_cores(
    patch: np.ndarray, 
    core_instances: np.ndarray, 
    core_label: int, 
    border_label: int, 
    min_distance: float = 1, 
    min_ratio_threshold: float = 0.95, 
    max_distance: float = 3, 
    max_ratio_threshold: float = 0.0) -> Tuple[np.ndarray, np.ndarray, int]:

    distances = distance_transform_edt(patch == core_label)
    core_ids = np.unique(core_instances)
    core_ids_to_remove = []
    for core_id in core_ids:
        core_distances = distances[core_instances == core_id]
        num_min_distances = np.count_nonzero(core_distances <= min_distance)
        num_max_distances = np.count_nonzero(core_distances >= max_distance)
        num_core_voxels = np.count_nonzero(core_instances == core_id)
        min_ratio = num_min_distances / num_core_voxels
        max_ratio = num_max_distances / num_core_voxels
        if (min_ratio_threshold is None or min_ratio >= min_ratio_threshold) and (max_ratio_threshold is None or max_ratio <= max_ratio_threshold):
            core_ids_to_remove.append(core_id)
    num_cores = len(core_ids) - len(core_ids_to_remove)
    if len(core_ids_to_remove) > 0:
        target_values = np.zeros_like(core_ids_to_remove, dtype=int)
        shape = patch.shape
        core_instances = npi.remap(core_instances.flatten(), core_ids_to_remove, target_values).reshape(shape)
        patch[(patch == core_label) & (core_instances == 0)] = border_label

    return patch, core_instances, num_cores

In [38]:
dir_location = 'refine'
_, _, bordercore_path, instance_path = setup_paths(dir_location, False)
# img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
#             '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
#             '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
# img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']
img_names = ['4_GenericD12']

for img_name in img_names:
    input_path = os.path.join(bordercore_path, img_name)
    output_path = os.path.join(instance_path, img_name)
    process_tiff_dir(input_path, output_path)

Paths set


Processing 4_GenericD12: 100%|██████████| 956/956 [00:02<00:00, 464.49slice/s]


Processing complete. Output saved to D:\Darren\Files\database\tablet_dataset\instance_temp\tiff\4_GenericD12


In [5]:
dir_location = 'refine'
_, _, bordercore_path, instance_path = setup_paths(dir_location, False)
# img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
#             '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
#             '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
# img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']
img_names = ['4_GenericD12']

for img_name in img_names:
    input_path = os.path.join(bordercore_path, img_name)
    output_path = os.path.join(instance_path, img_name)
    process_tiff_dir(input_path, output_path)

Paths set


Border-Core2Instance:   0%|          | 1/11657 [03:19<645:22:52, 199.33s/it]


KeyboardInterrupt: 

In [ ]:
dir_location = 'refine'
_, _, bordercore_path, instance_path = setup_paths(dir_location, False)
img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
            '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
            '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
# img_names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']

for img_name in img_names:
    input_path = os.path.join(bordercore_path, img_name)
    output_path = os.path.join(instance_path, img_name)
    process_tiff_dir(input_path, output_path)

# Code to Compare Two TIFF Stacks

In [34]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

def load_tiff(filepath):
    return np.array(Image.open(filepath))

def compare_dirs(dir1, dir2, diff_threshold=0):
    files1 = sorted([f for f in os.listdir(dir1) if f.endswith('.tif') or f.endswith('.tiff')])
    files2 = sorted([f for f in os.listdir(dir2) if f.endswith('.tif') or f.endswith('.tiff')])

    assert len(files1) == len(files2), "Number of slices do not match"
    
    total_mae = 0
    total_mse = 0
    total_diff_pixels = 0
    total_pixels = 0
    n = len(files1)

    for f1, f2 in tqdm(zip(files1, files2), total=n):
        img1 = load_tiff(os.path.join(dir1, f1)).astype(np.float32)
        img2 = load_tiff(os.path.join(dir2, f2)).astype(np.float32)

        assert img1.shape == img2.shape, f"Shape mismatch: {f1} vs {f2}"

        diff = np.abs(img1 - img2)
        diff_pixels = np.sum(diff > diff_threshold)
        num_pixels = diff.size

        total_mae += np.mean(diff)
        total_mse += np.mean(diff ** 2)
        total_diff_pixels += diff_pixels
        total_pixels += num_pixels

    avg_mae = total_mae / n
    avg_mse = total_mse / n
    diff_percent = (total_diff_pixels / total_pixels) * 100

    print(f"\nAverage MAE: {avg_mae:.4f}")
    print(f"Average MSE: {avg_mse:.4f}")
    print(f"Total different pixels: {total_diff_pixels} / {total_pixels} ({diff_percent:.2f}%)")

# Example usage
dir1 = r'd:\Darren\Files\database\tablet_dataset\instance\tiff\2_Tablet'
dir2 = r'd:\Darren\Files\database\tablet_dataset\instance_temp\tiff\2_Tablet'
compare_dirs(dir1, dir2)


  0%|          | 0/950 [00:00<?, ?it/s]

100%|██████████| 950/950 [00:12<00:00, 78.11it/s]


Average MAE: 0.0000
Average MSE: 0.0000
Total different pixels: 0 / 420822450 (0.00%)
